# Table of Contents
 <p>

Here we will try to give some enlighthenment on the influence of the parameters
given to Cundall detection method.

There are 3 directories in which different computation will take place.


In [ ]:

from pylmgc90 import chipy

### computation's parameters definition ### 

dt = 0.002
nb_steps = 3000
theta = 0.5

freq_display = 10

xperiode = 25.0
yperiode = 25.0


In [ ]:
workDir = 'CpCundall1'

cundall_it = 300
#cundall_it = 40

low_sap = 70

chipy.overall_SetWorkingDirectory(workDir)
chipy.PRPRx_UseCpCundallDetection(cundall_it)
chipy.PRPRx_LowSizeArrayPolyr(low_sap)

chipy.checkDirectories()

In [ ]:

freq_detect = 1
tol    = 0.1666e-3
relax  = 1.0
stype  = 'Stored_Delassus_Loops         '
quad   = 'Maxm '
gs_it1 = 10
gs_it2 = 100

#nlgs_3D_SetWithQuickScramble()
chipy.Initialize()
chipy.SetDimension(3)

chipy.utilities_logMes('INIT TIME STEPPING')
chipy.TimeEvolution_SetTimeStep(dt)
chipy.Integrator_InitTheta(theta)

chipy.RBDY3_NewRotationScheme()

### model reading ###
chipy.utilities_logMes('READ BODIES')
chipy.ReadBodies()

chipy.utilities_logMes('READ BEHAVIOURS')
chipy.ReadBehaviours()

#LOADS
chipy.LoadBehaviours()

chipy.utilities_logMes('READ INI DOF')
chipy.ReadIniDof()

chipy.LoadTactors()

chipy.utilities_logMes('READ INI Vloc Rloc')
chipy.ReadIniVlocRloc()

chipy.utilities_logMes('READ DRIVEN DOF')
chipy.ReadDrivenDof()

chipy.utilities_logMes('WRITE BODIES')
chipy.WriteBodies()
chipy.utilities_logMes('WRITE BEHAVIOURS')
chipy.WriteBehaviours()
chipy.utilities_logMes('WRITE DRIVEN DOF')
chipy.WriteDrivenDof()

### set periodic conditions ###
if workDir != 'CpCundallCluster':
    chipy.SetPeriodicCondition(xperiode,yperiode)

chipy.OpenDisplayFiles()
chipy.OpenPostproFiles()

### compute masses ###
chipy.ComputeMass()
if workDir == 'CpCundallCluster':
    RBDY3_PutBodyVector('Xbeg_', 3, [0., 0., 0.3, 0., 0., 0.])

for k in range(nb_steps):
    #
    chipy.IncrementStep()
    #
    #
    chipy.ComputeFext()
    chipy.ComputeBulk()
    chipy.ComputeFreeVelocity()
    #
    chipy.SelectProxTactors(freq_detect)

    chipy.RecupRloc()
    chipy.ExSolver(stype, quad, tol, relax, gs_it1, gs_it2)
    chipy.StockRloc()
    #
    chipy.ComputeDof()
    chipy.UpdateStep()

    chipy.WriteLastDof()
    chipy.WriteLastVlocRloc()

    chipy.WriteDisplayFiles(freq_display)
    chipy.WritePostproFiles()

    chipy.overall_CleanWriteOutFlags()

chipy.WriteLastDof()
chipy.WriteLastVlocRloc()

chipy.CloseDisplayFiles()
chipy.ClosePostproFiles()

chipy.Finalize()
